## Data wrangling

*Date:* 2024-07-17

- archive file downloaded from [Kaggle](https://www.kaggle.com/datasets/ishanshrivastava28/tata-online-retail-dataset)
- extracted `csv`file and copied to `analyses` folder
- perform wrangling operations to split off seed files for respective dimensions:
  - [customers](../seeds/01_ref_customers.csv): user IDs linked to shipping countries; some customers over time will have made orders to different shipping countries
  - [stock items](../seeds/02_ref_stock.csv): list of stock items and services rendered by the seller
  - [countries](../seeds/03_ref_countries.csv): most likely shipping destination enriched with continent / business unit information via labels by [ChatGPT](https://chat.openai.com/?sso=)

In [1]:
import pandas as pd

df = pd.read_csv('Online_Retail_Data_Set.csv', encoding='unicode_escape')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [3]:
# data type casting
#df.astype({'CustomerID':'Int64', 'b':np.float32})
df['CustomerID'] = df['CustomerID'].astype('Int64',)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format="%d-%m-%Y %H:%M")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      541909 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 33.6+ MB


In [4]:
df_columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

print("Listing Nulls")
for bla in df.columns:
  print(f"{bla}: {df[df[bla].isnull()].shape[0]}")

print("Listing NA")
for bla in df.columns:
  print(f"{bla}: {df[df[bla].isna()].shape[0]}")

print("Listing Missing Values")
print(f"Country: {df[df['Country']=='Unspecified'].shape[0]}")

Listing Nulls
InvoiceNo: 0
StockCode: 0
Description: 1454
Quantity: 0
InvoiceDate: 0
UnitPrice: 0
CustomerID: 135080
Country: 0
Listing NA
InvoiceNo: 0
StockCode: 0
Description: 1454
Quantity: 0
InvoiceDate: 0
UnitPrice: 0
CustomerID: 135080
Country: 0
Listing Missing Values
Country: 446


In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


## Customers

In [6]:
# copy and drop nulls
df_customer = df[['CustomerID', 'Country', 'InvoiceDate']].copy()
print(f"before drop: {df_customer.shape[0]}")
df_customer.dropna(inplace=True)
print(f"after drop: {df_customer.shape[0]}")

df_customer[df_customer['CustomerID'] == 17850]

before drop: 541909
after drop: 406829


,CustomerID,Country,InvoiceDate
0,17850,United Kingdom,2010-12-01 08:26:00
1,17850,United Kingdom,2010-12-01 08:26:00
2,17850,United Kingdom,2010-12-01 08:26:00
3,17850,United Kingdom,2010-12-01 08:26:00
4,17850,United Kingdom,2010-12-01 08:26:00
...,...,...,...
86898,17850,United Kingdom,2011-02-10 14:38:00
86899,17850,United Kingdom,2011-02-10 14:38:00
86900,17850,United Kingdom,2011-02-10 14:38:00
86901,17850,United Kingdom,2011-02-10 14:38:00


In [7]:
# drop general dupes
df_customer.sort_values(by=['CustomerID', 'InvoiceDate'], ascending=[True, True], inplace=True)
df_customer = df_customer.reset_index(drop=True)
print(f"after sort: {df_customer.shape[0]}")
df_customer.drop_duplicates(subset=['CustomerID', 'Country', 'InvoiceDate'], keep='first', inplace=True)
df_customer = df_customer.reset_index(drop=True)
print(f"after drop - unique customers/orders: {df_customer.shape[0]}")
df_customer[df_customer['CustomerID'] == 17850]


after sort: 406829
after drop - unique customers/orders: 22034


,CustomerID,Country,InvoiceDate
20542,17850,United Kingdom,2010-12-01 08:26:00
20543,17850,United Kingdom,2010-12-01 08:28:00
20544,17850,United Kingdom,2010-12-01 09:01:00
20545,17850,United Kingdom,2010-12-01 09:02:00
20546,17850,United Kingdom,2010-12-01 09:32:00
20547,17850,United Kingdom,2010-12-01 09:34:00
20548,17850,United Kingdom,2010-12-01 10:51:00
20549,17850,United Kingdom,2010-12-01 10:52:00
20550,17850,United Kingdom,2010-12-01 11:33:00
20551,17850,United Kingdom,2010-12-01 11:34:00


In [8]:
df_customer.to_csv('01_ref_customers.csv', index=False)

As can be seen above, all orders of each customer are still contained in the table.
Eventually, we will only need the shipping country changes and when they occurred.

Move `csv` file to dbt's `seeds` folder. Pandas would have been able to squash this data even more, but we want to let `dbt` take care of this.

Below is an example to do this with pandas instead.

In [9]:
# duplicate data
df_cust = df_customer.copy()

# don't use: sorts and drops islands
df_cust.drop_duplicates(subset=['CustomerID', 'Country'], keep='first', inplace=True)
print(f"after drop - simple method - customers: {df_cust.shape[0]}")

# reset --- duplicate data again
df_cust = df_customer.copy()

# window function
df_cust['CountryLag'] = df_cust.groupby(['CustomerID'])['Country'].shift(1)

# keep changes only
df_cust = df_cust[df_cust['CountryLag'] != df_cust['Country']]
print(f"after drop - elaborate method - customers: {df_cust.shape[0]}")

# verify/filter by duplicate per group
df_cust.groupby('CustomerID').filter(lambda x: len(x) > 1)

# ready for output
#df_cust = df_cust.drop(columns=['CountryLag'])
#df_cust = df_cust.reset_index(drop=True)


after drop - simple method - customers: 4380
after drop - elaborate method - customers: 4388


,CustomerID,Country,InvoiceDate,CountryLag
68,12370,Cyprus,2010-12-14 12:58:00,NaN
69,12370,Austria,2010-12-17 09:38:00,Cyprus
70,12370,Cyprus,2011-03-10 12:48:00,Austria
122,12394,Belgium,2011-05-06 14:01:00,NaN
123,12394,Denmark,2011-10-07 08:08:00,Belgium
214,12417,Belgium,2010-12-17 11:51:00,NaN
219,12417,Spain,2011-04-28 13:09:00,Belgium
220,12417,Belgium,2011-06-05 15:46:00,Spain
233,12422,Australia,2011-01-19 09:13:00,NaN
234,12422,Switzerland,2011-06-24 13:20:00,Australia


## Stock Items

In [10]:
# copy and drop nulls
df_stock = df[['StockCode', 'Description']].copy()
print(f"before drop: {df_stock.shape[0]}")
df_stock.dropna(inplace=True)
print(f"after drop: {df_stock.shape[0]}")

before drop: 541909
after drop: 540455


In [11]:
# remove leading/trailing blanks
df_stock['StockCode'] = df_stock['StockCode'].str.strip()
# uppercase
df_stock['StockCode'] = df_stock['StockCode'].str.upper()

# remove leading/trailing blanks
df_stock['Description'] = df_stock['Description'].str.strip()
# remove leading/trailing special chars
df_stock['Description'] = df_stock['Description'].str.strip('.,:;')
# uppercase
df_stock['Description'] = df_stock['Description'].str.upper()
df_stock

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART
...,...,...
541904,22613,PACK OF 20 SPACEBOY NAPKINS
541905,22899,CHILDREN'S APRON DOLLY GIRL
541906,23254,CHILDRENS CUTLERY DOLLY GIRL
541907,23255,CHILDRENS CUTLERY CIRCUS PARADE


As it turns out the data also contains transactions concerning stock management,
which should not be part of sales reporting.

For that reason it was decided to drop these information.

In [13]:
# drop general dupes
df_stock.drop_duplicates(subset=['StockCode', 'Description'], keep='first', inplace=True)
# calc length of description
df_stock['DescLength'] = df_stock['Description'].str.len()
# sort by code and desc length
df_stock.sort_values(by=['StockCode', 'DescLength'], ascending=[True, False], inplace=True)
# reindex
df_stock = df_stock.reset_index(drop=True)
df_stock['StockID'] = df_stock.index

# add flags for manual inspection
df_stock['HasDuplicate'] = df_stock.duplicated(subset=['StockCode'], keep=False)
df_stock['IsMultiple'] = df_stock.duplicated(subset=['StockCode'], keep='first')

df_stock = df_stock[['StockID', 'StockCode', 'Description', 'DescLength', 'HasDuplicate', 'IsMultiple']]
print(f"after first de-duplicate: {df_stock.shape[0]}")
df_stock

after first de-duplicate: 4672


,StockID,StockCode,Description,DescLength,HasDuplicate,IsMultiple
0,0,10002,INFLATABLE POLITICAL GLOBE,26,False,False
1,1,10080,GROOVY CACTUS INFLATABLE,24,True,False
2,2,10080,CHECK,5,True,True
3,3,10120,DOGGY RUBBER,12,False,False
4,4,10123C,HEARTS WRAPPING TAPE,20,False,False
...,...,...,...,...,...,...
4667,4667,GIFT_0001_50,DOTCOMGIFTSHOP GIFT VOUCHER £50.00,34,False,False
4668,4668,M,MANUAL,6,False,False
4669,4669,PADS,PADS TO MATCH ALL CUSHIONS,26,False,False
4670,4670,POST,POSTAGE,7,False,False


In [14]:
# replace chars:
# commas
#df_stock[df_stock['Description'].str.contains(',')]['Description'].str.replace(pat=',', repl=' - ')
# double quotes
#df_stock[df_stock['Description'].str.contains('"')]['Description'].str.replace(pat='"', repl='`')
# single quotes
#df_stock[df_stock['Description'].str.contains("'")]['Description'].str.replace(pat="'", repl='`')
# triple blanks
#df_stock[df_stock['Description'].str.contains('   ')]['Description'].str.replace(pat='   ', repl=' ')
# double blanks
#df_stock[df_stock['Description'].str.contains('  ')]['Description'].str.replace(pat='  ', repl=' ')

# commas
df_stock['Description'] = df_stock['Description'].str.replace(pat=',', repl=' - ')
# double quotes
df_stock['Description'] = df_stock['Description'].str.replace(pat='"', repl='`')
# single quotes
df_stock['Description'] = df_stock['Description'].str.replace(pat="'", repl='`')
# triple blanks
df_stock['Description'] = df_stock['Description'].str.replace(pat='   ', repl=' ')
# double blanks
df_stock['Description'] = df_stock['Description'].str.replace(pat='  ', repl=' ')


In [15]:
df_stock.to_csv('stock_inflated.csv', index=False)

Finally, the data needs to be de-duplicated, manually, outside of pandas:
- import to spreadsheet tool for manual inspection,
- eliminate all duplicate rows that don't describe the product,
- eliminate all singular rows that don't describe any service,
- there should only be one stock code per relevant service/product,
- move `csv` file to dbt's `seeds` folder.

## Countries

In [16]:
# copy and drop nulls
df_country = df[['Country', 'CustomerID']].copy()
print(f"before drop: {df_country.shape[0]}")
df_country.dropna(inplace=True)
print(f"after drop: {df_country.shape[0]}")

before drop: 541909
after drop: 406829


In [17]:
# drop general dupes
df_country.drop_duplicates(subset=['Country'], keep='first', inplace=True)
print(f"after de-duplication: {df_country.shape[0]}")
# sort by country
df_country.sort_values(by=['Country'], ascending=[True], inplace=True)
# reindex
df_country = df_country.reset_index(drop=True)
df_country['CountryID'] = df_country.index

# ready for output
df_country = df_country.drop(columns=['CustomerID'])
df_country = df_country[['CountryID', 'Country']]
df_country

after de-duplication: 37


,CountryID,Country
0,0,Australia
1,1,Austria
2,2,Bahrain
3,3,Belgium
4,4,Brazil
5,5,Canada
6,6,Channel Islands
7,7,Cyprus
8,8,Czech Republic
9,9,Denmark


In [18]:
df_country.to_csv('countries_unlabeled.csv', index=False)

As mention above:
- the shipping country data should be either labeled manually for *business unit* and *continent*.
- alternatively, use an LLM, e.g., ChatGPT for the chore,
- move `csv` file to dbt's `seeds` folder.